# Detailing thinking

In this case we will try to add tags and hoping that the model thinks before answering so it has a better response.

In [1]:
import openai
from openai import OpenAI
import json
import re
import helper.creden as cre
client = OpenAI(api_key=cre.OPENAI_API_KEY)
from helper.helper import parse_conversation, select_random_test, select_random_topics, get_completion_from_messages

# widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

client = OpenAI(api_key=cre.OPENAI_API_KEY)
model_selection="gpt-4o-mini"

In [2]:
messages_verification=[{"role": "user", "content": "Are you working?"}]
result = get_completion_from_messages(messages = messages_verification, model = model_selection, temperature=0.8, max_tokens_values=4)
result

"Yes, I'm here"

Now that we have confirmed the API is working, we will conduct an experiment.

In [3]:
tests_interviews = {
                "topics": [
                    {
                    "name": "Predictive Modeling",
                    "example": "Predicting customer churn for a subscription service.",
                    "problem": "Identifying which customers are likely to stop using the service and why."
                    },
                    {
                    "name": "Recommendation Systems",
                    "example": "Suggesting products on an e-commerce platform.",
                    "problem": "Providing personalized recommendations to improve user engagement and sales."
                    },
                    {
                    "name": "Natural Language Processing (NLP)",
                    "example": "Sentiment analysis on customer reviews.",
                    "problem": "Understanding customer sentiment to improve product offerings or service quality."
                    },
                    {
                    "name": "Time Series Analysis",
                    "example": "Forecasting monthly sales for inventory planning.",
                    "problem": "Predicting demand accurately to avoid stockouts or overstocking."
                    },
                    {
                    "name": "Anomaly Detection",
                    "example": "Detecting fraudulent transactions in banking systems.",
                    "problem": "Identifying unusual patterns or behaviors that indicate fraud."
                    },
                    {
                    "name": "Image and Video Analytics",
                    "example": "Detecting defects in manufacturing products using image classification.",
                    "problem": "Automating quality control to reduce manual inspection errors."
                    },
                    {
                    "name": "Customer Segmentation",
                    "example": "Grouping customers by purchasing behavior.",
                    "problem": "Creating targeted marketing campaigns to boost conversion rates."
                    },
                    {
                    "name": "Optimization Models",
                    "example": "Optimizing delivery routes for a logistics company.",
                    "problem": "Minimizing fuel costs and delivery time."
                    },
                    {
                    "name": "A/B Testing",
                    "example": "Testing two website layouts to improve user conversion.",
                    "problem": "Determining which design leads to better engagement or sales."
                    },
                    {
                    "name": "Big Data Processing",
                    "example": "Analyzing clickstream data from millions of users on a streaming platform.",
                    "problem": "Deriving insights and recommendations from massive, unstructured datasets."
                    }
                ]
                }
# select a random test
random_topic = select_random_test(tests_interviews)
random_topic


data_science_topics = [
    "Data Preprocessing", "Exploratory Data Analysis (EDA)", "Feature Engineering", "Statistical Analysis",
    "Probability Theory", "Supervised Learning", "Unsupervised Learning", "Model Evaluation Metrics",
    "Cross-Validation Techniques", "Overfitting and Underfitting", "Bias-Variance Tradeoff", 
    "Ensemble Learning", "Neural Networks", "Natural Language Processing (NLP)",
    "Time Series Analysis", "Spark", "Model Deployment and Monitoring", 
    "Reinforcement Learning", "Data Ethics and Privacy", "Cloud Computing for Data Science"
]

selected_topics = select_random_topics(data_science_topics)
selected_topics 

['Model Deployment and Monitoring',
 'Exploratory Data Analysis (EDA)',
 'Probability Theory',
 'Supervised Learning',
 'Ensemble Learning']

# Prompt generation

In [11]:
prompt_example = f"""
When you reply, first plan how you should answer within <thinking> </thinking> XML tags. this is a space for you to write down relevant content and will be be showing to the used, for now.
once you are done with the thinking you can output the answer of the thinking <answer> </answer> XML tags. Make sure your answer is detailed and specific  
We are conducting a data science interview focused exclusively on 
data science topics. Avoid discussing unrelated subjects. Ensure
a rigorous and structured interview process with the goal of 
evaluating the candidate's expertise in data science. 
for each well-answered question, and maintain high 
expectations as a data science professional with 5 years of
experience. Look for key terminology and concepts relevant
to the field.

---------------
Generated Knowledge Example Question: 
- We are working with a healthcare company to predict the likelihood of patients being readmitted within 30 days of discharge. The dataset includes patient demographics, medical history, and various other details related to their hospital stay. What steps would you take to build a model for this prediction task?
- You’ve chosen a random forest model for predicting house prices. Could you explain what might happen if the model starts overfitting the data? What steps would you take to mitigate overfitting in this case, and how would you determine if the model is truly generalizing well?

---------------
<question>
Steps to Follow:
Introduction: Begin with a brief introduction of the company. Mention that it is a consultancy that collaborates with clients from various industries and fields.
Questioning: Ask one question at a time, ensuring it pertains to data science. Move on to the next question only after receiving an answer. ask just 3 question in each interview as much
Example Problem: Incorporate this field example {random_topic["example"]} and this problem {random_topic["problem"]} as a case study to keep the discussion grounded, also include this topics in the questions {selected_topics}.
Dataset Creation: Design a simple dataset with 5 rows and 4–8 features, including a target variable, to use as part of the problem-solving exercise. Present this dataset in a table format.
this its the example of the dataset
Question Design: Formulate one thoughtful question at a time based on the dataset or case study. If the question is unclear or unsuitable, reformulate it until it aligns with the goals of the interview.
Scoring: Assign a score from 0 to 9 for each response, where 9 represents expert-level knowledge. Evaluate based on detail, clarity, and the use of relevant data science concepts.
Conclusion: Conclude by thanking the candidate for their time and providing an overall score based on their performance.
Feedback: After completing all questions, identify areas where the interview process could be improved, providing specific points for enhancement.
Final Output: Summarize the results in a JSON format, including the overall score and interview feedback.
</question>
---------------
Interview Scoring System:
Each question is scored from 0 to 9, where 9 reflects expert-level proficiency. Evaluate the answers based on:
Clarity: Was the answer well-structured and easy to follow?
Detail: Did the candidate demonstrate depth in their understanding of data science principles?
Application of Concepts: Did the candidate incorporate relevant data science terminology and concepts (e.g., feature engineering, model selection, evaluation metrics)?
Example: "7/9" or "0/9".

---------------
before ending the chat add a JSON using this format
<Example>
Extracted Data:
[
  "candidate_name": "Jackeline",
  "scores": [
    "question_1": 5,
    "question_2": 6,
    "question_3": 0
  ],
  "overall_score": 11,
  "feedback": [
    "Depth of Knowledge: While you demonstrated some understanding of data science concepts, there is a need for deeper knowledge in model evaluation and data preprocessing techniques.",
    "Specificity: Providing more detailed explanations and examples would enhance your responses.",
    "Preparation: Familiarizing yourself with key evaluation metrics and their implications in classification problems would be beneficial."
  ]
]
</Example>

Reminder: Ensure the questions and evaluations are rigorous, leveraging your data science expertise to assess technical depth and problem-solving abilities.

After printing the json just keep saying good bye in case the person wants to keep talking.
"""

context = [{'role':'system', 'content':prompt_example}]
context

[{'role': 'system',
  'content': '\nWhen you reply, first plan how you should answer within <thinking> </thinking> XML tags. this is a space for you to write down relevant content and will be be showing to the used, for now.\nonce you are done with the thinking you can output the answer of the thinking <answer> </answer> XML tags. Make sure your answer is detailed and specific  \nWe are conducting a data science interview focused exclusively on \ndata science topics. Avoid discussing unrelated subjects. Ensure\na rigorous and structured interview process with the goal of \nevaluating the candidate\'s expertise in data science. \nfor each well-answered question, and maintain high \nexpectations as a data science professional with 5 years of\nexperience. Look for key terminology and concepts relevant\nto the field.\n\n---------------\nGenerated Knowledge Example Question: \n- We are working with a healthcare company to predict the likelihood of patients being readmitted within 30 days of

We use, Generated Knowledge	when used to create structured, expert-level questions relevant to data science.
But Maieutic Prompting, it’s used in the iterative refinement of responses, ensuring high-quality evaluation.

In [12]:
# Create text box for user input
user_input = widgets.Textarea(value='',
                              placeholder='Enter text here...',
                              description='User:',
                              rows=2,
                              layout=widgets.Layout(width='auto'),
                              disabled=False)

# Create text area for conversation display
conversation_history = widgets.Textarea(value='',
                                        description='Chat:',
                                     rows=15,
                                        layout=widgets.Layout(width='auto'))

# Create button for user to click to send their message
send_button = widgets.Button(description='Send')

# Initial Question (without response)
initial_question = "We will start with the interview. Can you give me a brief introduction about yourself?"
conversation_history.value += 'assistant: ' + initial_question + '\n'
context.append({'role': 'assistant', 'content': initial_question})

# -------------------------------
def send_button_clicked(b):
    """Send content to the chat"""
    global context
    
    conversation_history.value += 'user: ' + user_input.value + '\n'
    context.append({'role': 'user', 'content': user_input.value})
    response = get_completion_from_messages(messages = context, model = model_selection, temperature=0.1, max_tokens_values=2000)
    conversation_history.value += 'assistant: ' + response + '\n'
    context.append({'role': 'assistant', 'content': response})
    user_input.value = ''
    
# -------------------------------
send_button.on_click(send_button_clicked)

# -------------------------------
# Add keyboard shortcut for sending (Shift+Enter)
def handle_keypress(widget):
    if widget.value.endswith('\n'):
        send_button_clicked(None)
        
user_input.observe(handle_keypress, names='value')
# -------------------------------
# Display everything
display(conversation_history, user_input, send_button)

Textarea(value='assistant: We will start with the interview. Can you give me a brief introduction about yourse…

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='Send', style=ButtonStyle())

# Review of the interaction

In [13]:
json_result = parse_conversation(conversation_history.value)
print(json.dumps(json_result, indent=2))

{
  "conversation": [
    {
      "role": "assistant",
      "content": "We will start with the interview. Can you give me a brief introduction about yourself?"
    },
    {
      "role": "user",
      "content": "My name its jorge"
    },
    {
      "role": "assistant",
      "content": "<thinking>\nI will now proceed to ask the first question related to data science, specifically focusing on the case study of optimizing delivery routes for a logistics company. The question will involve exploratory data analysis (EDA) and model deployment. I will ensure that the question is clear and relevant to the candidate's experience level.\n</thinking>\n<answer>\nThank you for your introduction, Jorge. Let's dive into the first question.\nImagine you are tasked with optimizing delivery routes for a logistics company to minimize fuel costs and delivery time. You have access to a dataset that includes features such as delivery distance, traffic conditions, vehicle type, and delivery time. \n1. Wh

In [15]:
# pattern = r'```json\s*(\{.*?\})\s*```'
pattern = r'<Example>\s*Extracted Data:\s*(\[[\s\S]*?\])\s*</Example>'
match = re.search(pattern, conversation_history.value, re.DOTALL)

if match:
    extracted_data = match.group(1)
    print(f"Extracted Data:\n{extracted_data}")
else:
    print("No match found.")

Extracted Data:
[
  "candidate_name": "Jorge",
  "scores": [
    "question_1": 5,
    "question_2": 6,
    "question_3": 4
  ],
  "overall_score": 15,
  "feedback": [
    "Depth of Knowledge: You demonstrated a basic understanding of data science concepts, but there is room for improvement in providing detailed explanations.",
    "Specificity: Your answers could benefit from more specific examples and a deeper exploration of the metrics and techniques relevant to the questions.",
    "Preparation: Familiarizing yourself with a broader range of evaluation metrics and their implications would enhance your responses."
  ]
]


The adding of tags helps improve the logic of the LLM adding following questions when we need it, it 
also seems that we could understand better the deposition making of the model,
in this case the answers need to be longer so we could get more point, but seems tat its working as expected, so we will using this prompt for the model deployment.